In [19]:
import pandas as pd
import numpy as np
import joblib
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [20]:
data_path = r'C:\Users\azizh\OneDrive\Bureau\MedCareAI\ModelReco.ipynb\data_reco_cleaned.csv'
data = pd.read_csv(data_path)

data.head()

,disease,severity_score,recommended_medication,alternative_medication_1,alternative_medication_2,symptom_count,requires_surgery,patient_notes,medicine_usage_notes,contagious
0,cystitis,7,Antibiotic,Painkiller,Topical cream,4,False,Patient reports burning sensation.,Take 1 tablet every 8 hours.,False
1,vulvodynia,5,Painkiller,Anti-inflammatory,Topical cream,3,False,Symptoms worsening over 2 days.,Apply cream twice daily.,False
2,nose disorder,3,Antihistamine,Painkiller,Anti-inflammatory,2,False,"Mild congestion, no fever.",1 tablet daily after meal.,True
3,complex regional pain syndrome,8,Painkiller,Anti-inflammatory,Topical cream,5,True,"Severe pain in limb, mobility limited.",Take 2 tablets daily.,False
4,spondylosis,6,Anti-inflammatory,Painkiller,Topical cream,4,True,"Chronic back pain, stiffness.",Apply cream on affected area.,False


In [21]:
data.columns.tolist()

['disease',
 'severity_score',
 'recommended_medication',
 'alternative_medication_1',
 'alternative_medication_2',
 'symptom_count',
 'requires_surgery',
 'patient_notes',
 'medicine_usage_notes',
 'contagious']

In [22]:
X = data[['disease', 'severity_score', 'symptom_count', 'requires_surgery']].copy()

# Encode categorical feature 'disease'
le_disease = LabelEncoder()
X['disease'] = le_disease.fit_transform(X['disease'])

In [23]:
y = data[['recommended_medication', 'alternative_medication_1', 'alternative_medication_2', 'medicine_usage_notes']].copy()

# Encode each target column
target_encoders = {}
for col in y.columns:
    le = LabelEncoder()
    y[col] = le.fit_transform(y[col])
    target_encoders[col] = le

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
model = MultiOutputClassifier(RandomForestClassifier(n_estimators=200, random_state=42))
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

C:\Users\azizh\AppData\Roaming\Python\Python311\site-packages\sklearn\multioutput.py:245: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  check_classification_targets(y)
C:\Users\azizh\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_forest.py:830: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  check_classification_targets(y)
C:\Users\azizh\AppData\Roaming\Python\Python311\site-packages\sklearn\tree\_classes.py:297: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  check_classification_targets(y)
C:\Users\azizh\AppData\Roaming\Python\Python311\site-packages\sklearn\tree\_classes.py:297: UserWarning: The number of unique classes is grea

In [26]:
# Decode predictions back to text
y_pred_decoded = pd.DataFrame()
for col in y.columns:
    y_pred_decoded[col] = target_encoders[col].inverse_transform(y_pred[:, y.columns.get_loc(col)])

In [29]:
from sklearn.metrics import accuracy_score

for col in y.columns:
    acc = accuracy_score(y_test[col], y_pred[:, y.columns.get_loc(col)])
    print(f"Accuracy for {col}: {acc:.2f}")

Accuracy for recommended_medication: 0.25
Accuracy for alternative_medication_1: 0.36
Accuracy for alternative_medication_2: 0.18
Accuracy for medicine_usage_notes: 0.07


In [30]:
new_patient = pd.DataFrame({
    'disease': le_disease.transform(['cystitis']),
    'severity_score': [7],
    'symptom_count': [4],
    'requires_surgery': [0]
})

prediction = model.predict(new_patient)
pred_decoded = {col: target_encoders[col].inverse_transform([prediction[0, i]])[0] for i, col in enumerate(y.columns)}

print("\nRecommended medicine for new patient:")
print(pred_decoded)


Recommended medicine for new patient:
{'recommended_medication': 'Antibiotic', 'alternative_medication_1': 'Painkiller', 'alternative_medication_2': 'Topical cream', 'medicine_usage_notes': 'Take 1 tablet every 8 hours.'}


In [31]:
import joblib

# Enregistrer le modèle
joblib.dump(model, "medic_recommendation_model.pkl")

# Enregistrer aussi les encodeurs pour décoder les labels
joblib.dump(le_disease, "disease_encoder_for_reco.pkl")
joblib.dump(target_encoders, "target_encoders_for_reco.pkl")

print("Modèle et encodeurs sauvegardés avec succès !")


Modèle et encodeurs sauvegardés avec succès !
